In [12]:
top_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'

import os
import sys
import time
from math import *
import copy
import cPickle as pickle

# data
import numpy as np
import pandas as pd

# viz
import matplotlib.pyplot as plt


# graph
import igraph as ig


# our code
sys.path.append(top_directory + 'code/')
from pipeline.download_data import *
from pipeline.make_raw_case_metadata import *
from helpful_functions import case_info

sys.path.append(top_directory + 'explore/vertex_metrics_experiment/code/')
from make_edge_df import *
from rankscore_experiment_sort import *
from rankscore_experiment_LR import *


# court
network_name = 'scotus'

# directory set up
data_dir = '/Users/iaincarmichael/Documents/courtlistener/data/'
subnet_dir = data_dir + 'scotus/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'



# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
G = ig.Graph.Read_GraphML(subnet_dir + 'scotus_network.graphml')

In [8]:
metrics = ['indegree', 'outdegree', 'degree', 'd_pagerank', 'authorities', 'hubs', 'similarity']


test_params = {'active_years':  range(1900, 2015 + 1),
               'seed': 4332,
               'num_test_cases': 10}

In [159]:
scores_sort = get_rankscores_sort(G, test_params, metrics,
                             subnet_dir,
                             print_progress=True)

# logistic regression

In [9]:
num_non_edges_to_add = len(G.es)

seed_edge_df = 32432
# active_years = range(1900, 2015 + 1)

active_years = range(1950, 1980 + 1)

include_similarity = True

metric_normalization = 'mean'

In [291]:
%time make_edge_df(G, subnet_dir, \
                   active_years, num_non_edges_to_add,\
                   columns_to_use=metrics, metric_normalization=metric_normalization,\
                   seed=seed_edge_df)

CPU times: user 12min 18s, sys: 18.9 s, total: 12min 37s
Wall time: 12min 38s


In [13]:
%time scores_LR = get_rankscores_LR(G, test_params, metrics, \
                                    include_similarity, subnet_dir,\
                                    metric_normalization=metric_normalization, print_progress=True) 
                                    

indegree
(1/10) at 22:11:06
(2/10) at 22:11:06
(4/10) at 22:11:07
(8/10) at 22:11:08
outdegree
(1/10) at 22:11:10
(2/10) at 22:11:10
(4/10) at 22:11:11
(8/10) at 22:11:12
degree
(1/10) at 22:11:14
(2/10) at 22:11:14
(4/10) at 22:11:15
(8/10) at 22:11:16
d_pagerank
(1/10) at 22:11:17
(2/10) at 22:11:18
(4/10) at 22:11:18
(8/10) at 22:11:20
authorities
(1/10) at 22:11:21
(2/10) at 22:11:21
(4/10) at 22:11:22
(8/10) at 22:11:24
hubs
(1/10) at 22:11:25
(2/10) at 22:11:26
(4/10) at 22:11:27
(8/10) at 22:11:28
similarity
(1/10) at 22:11:29
(2/10) at 22:11:30
(4/10) at 22:11:30
(8/10) at 22:11:32
CPU times: user 42.7 s, sys: 2.65 s, total: 45.3 s
Wall time: 33.7 s


# scaling